In [64]:
from random import randint, random
from matplotlib import pyplot as plt

In [48]:
class Individuo:
  def __init__(self, n_genes, v_min, v_max):
    self.cromossomo = self.iniciar_individuo(n_genes, v_min, v_max)
    self.fitness = self.get_fitness()

  def ajustar_cromossomo(self, cromossomo):
    self.cromossomo = cromossomo
    self.fitness = self.get_fitness()

  def atualizar_aptidao(self):
    self.fitness = self.get_fitness()

  def __str__(self) -> str:
    return f'Cromossomo: {self.cromossomo}, Fitness: {self.fitness}'

  def iniciar_individuo(self, n_genes, min_v, max_v):
    vetor = []

    for i in range(n_genes):
      gene = randint(min_v, max_v)
      vetor.append(gene)
    return vetor

  def get_fitness(self):
    # função sphere
    return sum(x**2 for x in self.cromossomo)

In [49]:
def iniciar_populacao(n_individuos, n_genes, min_v, max_v):
  populacao = []

  for i in range(0, n_individuos):
    individuo = Individuo(n_genes, min_v, max_v)
    populacao.append(individuo)
  return populacao


In [45]:
def imprimir_populacao(populacao):
  for individuo in populacao:
    print(f'{individuo.cromossomo} - {individuo.fitness}')

In [46]:
# abordagem de seleção por torneio
def selecao_torneio(populacao, n_individuos):
  indice_melhor_torneio = -1
  fitness_melhor_torneio = -1

  for i in range(0, n_individuos):
    x = randint(0, len(populacao) -1)

    print(f'indices sorteados: {x}, populaçao: {populacao[x].fitness}')

    if indice_melhor_torneio == -1 or \
      populacao[x].fitness < fitness_melhor_torneio:

      indice_melhor_torneio = x
      fitness_melhor_torneio = populacao[x].fitness

  return indice_melhor_torneio



In [50]:
def cruzamento_uniforme(individuo1, individuo2, n_genes, min_v, max_v):

  filho1 = Individuo(n_genes, min_v, max_v)
  filho2 = Individuo(n_genes, min_v, max_v)

  filho1_cr = []
  filho2_cr = []

  for i in range(0, n_genes):
    selecao = randint(1,2)
    if selecao == 1:
      filho1_cr.append(individuo1.cromossomo[i])
      filho2_cr.append(individuo2.cromossomo[i])
    else:
      filho1_cr.append(individuo2.cromossomo[i])
      filho2_cr.append(individuo1.cromossomo[i])

  filho1.ajustar_cromossomo(filho1_cr)
  filho2.ajustar_cromossomo(filho2_cr)

  print(f'PAI1 {individuo1} x PAI2 {individuo2}')
  print(f'FILHO1 {filho1} x FILHO2 {filho2}')

  return filho1, filho2


In [ ]:
def mutacao_uniforme(individuo, n_genes, min_v, max_v, taxa_mutacao):
  prob = random()
  novo_cromossomo = []

  for i in range(0, n_genes):
    if prob < taxa_mutacao:
      novo_valor = randint(min_v, max_v)
      novo_cromossomo.append(novo_valor)
      print(f'Mutou!{i}')
    else:
      novo_cromossomo.append(individuo.cromossomo[i])

  print(f'individuo antes da mutação <{individuo}>')
  individuo.ajustar_cromossomo(novo_cromossomo)
  print(f'individuo depois da mutação <{individuo}>')
  return individuo

In [56]:
def melhor_individuo(populacao):
  # return int(min(populacao, key=lambda x: x.fitness))
  melhor_fit = populacao[0].fitness
  for individuo in populacao:
    if individuo.fitness < melhor_fit:
      melhor_fit = individuo.fitness
  return melhor_fit

In [74]:
def algoritmo_genetico(n_geracoes, n_populacao, n_genes, min_v, max_v, n_individuos_torneio, taxa_mutacao):
    populacao = iniciar_populacao(n_populacao, n_genes, min_v, max_v)
    melhor_individuo_atual = melhor_individuo(populacao)
    vetor_solucao = []

    for contador_geracoes in range(n_geracoes):
        nova_populacao = []

        for i in range(0, n_populacao, 2):
            # Seleção para cruzamento (torneio)
            ind_pai1 = selecao_torneio(populacao, n_individuos_torneio)
            ind_pai2 = selecao_torneio(populacao, n_individuos_torneio)

            pai1 = populacao[ind_pai1]
            pai2 = populacao[ind_pai2]

            # Cruzamento uniforme
            filho1, filho2 = cruzamento_uniforme(pai1, pai2, n_genes, min_v, max_v)

            # Mutação uniforme
            filho1 = mutacao_uniforme(filho1, n_genes, min_v, max_v, taxa_mutacao)
            filho2 = mutacao_uniforme(filho2, n_genes, min_v, max_v, taxa_mutacao)

            # Adicionar filhos à nova população
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)

        # A população é completamente substituída pela nova geração (sem elitismo)
        populacao = nova_populacao

        melhor_individuo_geracao = melhor_individuo(populacao)

        # Problema de minimização: atualizar o melhor indivíduo se necessário
        if melhor_individuo_geracao < melhor_individuo_atual:
            melhor_individuo_atual = melhor_individuo_geracao
        vetor_solucao.append(melhor_individuo_geracao)

        print(f'[{contador_geracoes}: {str(melhor_individuo_geracao)} x {str(melhor_individuo_atual)}]')

    return melhor_individuo_atual, vetor_solucao


In [78]:
def executar_algoritmo_genetico(n_execucoes, n_geracoes, n_populacao, n_genes, min_v, max_v, n_individuos_torneio, taxa_mutacao):
  for exec in range(0, n_execucoes):

    melhor_individuo, vetor_solucoes = algoritmo_genetico(
      n_geracoes,
      n_populacao,
      n_genes,
      min_v,
      max_v,
      n_individuos_torneio,
      taxa_mutacao
    )
    if exec == 0:
      evolucao_media = vetor_solucoes

    for i in range(0, n_geracoes):
      evolucao_media[i] = evolucao_media[i] + vetor_solucoes[i]

  # media das evoluções do ag ao longo das n_execuções
  for i in range(0, n_geracoes):
    evolucao_media[i] = evolucao_media[i] / n_execucoes

  print(f'evolução média: {evolucao_media}')

  return evolucao_media




In [ ]:
# teste separado
populacao = iniciar_populacao(10, 5, -10, 10)
imprimir_populacao(populacao)
ind1 = selecao_torneio(populacao, 3)
ind2 = selecao_torneio(populacao, 3)
print('-------------------------------------------------')
filho1, filho2 = cruzamento_uniforme(populacao[ind1], populacao[ind2], 5, -10, 10)
mutacao_uniforme(filho1, 5, -10, 10, 0.5)
mutacao_uniforme(filho2, 5, -10, 10, 0.5)
print('-------------------------------------------------')
print(f'melhor individuo> {str(melhor_individuo(populacao))}')

In [ ]:
# teste aplicando o algoritmo
n_pop = 30
n_genes = 5
min_v = -5
max_v = 5
qt_individuos_torneio = 3
tx_mut = 0.05
n_geracoes = 100
n_execucoes = 5
sol1 = executar_algoritmo_genetico(
    n_execucoes,
    n_geracoes,
    n_pop,
    n_genes,
    min_v,
    max_v,
    qt_individuos_torneio,
    tx_mut
)
sol2 = executar_algoritmo_genetico(
    n_execucoes,
    n_geracoes,
    n_pop,
    n_genes,
    min_v,
    max_v,
    qt_individuos_torneio,
    (tx_mut+ 0.02)
)
plt.plot(sol1)
plt.plot(sol2)